In [1]:
from pathlib import Path
import numpy as np
import torch
from typing import List
from torch.nn.utils.rnn import pad_sequence
from mltrainer import rnn_models, Trainer
from torch import optim

from mads_datasets import datatools
import mltrainer
mltrainer.__version__

'0.2.5'

# 1 Iterators
We will be using an interesting dataset. [link](https://tev.fbk.eu/resources/smartwatch)

From the site:
> The SmartWatch Gestures Dataset has been collected to evaluate several gesture recognition algorithms for interacting with mobile applications using arm gestures. Eight different users performed twenty repetitions of twenty different gestures, for a total of 3200 sequences. Each sequence contains acceleration data from the 3-axis accelerometer of a first generation Sony SmartWatch™, as well as timestamps from the different clock sources available on an Android device. The smartwatch was worn on the user's right wrist. 


In [2]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2025-10-13 21:22:58.557 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\tycoh\.cache\mads_datasets\gestures
100%|██████████| 651/651 [00:11<00:00, 55.59it/s]


In [3]:
len(train), len(valid)

(81, 20)

In [31]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))
x.shape, y

(torch.Size([32, 34, 3]),
 tensor([ 9, 10, 14,  7, 13, 16, 12, 11, 19, 17,  5,  0, 15, 16, 14, 19, 12, 14,
          7, 17,  5,  9, 14,  9, 11,  6, 17,  3,  7, 11,  5,  3]))

Can you make sense of the shape?
What does it mean that the shapes are sometimes (32, 27, 3), but a second time might look like (32, 30, 3)? In other words, the second (or first, if you insist on starting at 0) dimension changes. Why is that? How does the model handle this? Do you think this is already padded, or still has to be padded?

- So it is 32 batches, 37 timesteps and 3 features or it could be 32 batches 30 timesteps and 3 features. The variance in timesteps depends on the length of the gesture. The model cannot handle this so therefore you need padding in order to have all the same size of timesteps.

# 2 Excercises
Lets test a basemodel, and try to improve upon that.

Fill the gestures.gin file with relevant settings for `input_size`, `hidden_size`, `num_layers` and `horizon` (which, in our case, will be the number of classes...)

As a rule of thumbs: start lower than you expect to need!

In [26]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

accuracy = Accuracy()


In [32]:
model = rnn_models.BaseRNN(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    horizon=20,
)

Test the model. What is the output shape you need? Remember, we are doing classification!
- We need a 2D output tensor because we want to classify the gestures and a fully connected layers needs a 2D shape (batches and class)

In [36]:
yhat = model(x)
yhat.shape

torch.Size([32, 20])

Test the accuracy

In [37]:
accuracy(y, yhat)

0.09375

What do you think of the accuracy? What would you expect from blind guessing?
- This is a very bad accuracy, only 9% of the time it is correct. I would expect an accuracy of 1/20 from blind guessing.

Check shape of `y` and `yhat`

In [9]:
yhat.shape, y.shape

(torch.Size([32, 20]), torch.Size([32]))

And look at the output of yhat

In [45]:
yhat[3]

tensor([-0.0323, -0.1437,  0.1171,  0.1633,  0.0516, -0.0692, -0.1154, -0.0357,
         0.0878,  0.0021, -0.0021,  0.2095, -0.0472, -0.1498, -0.0840,  0.0021,
        -0.0688,  0.0677, -0.0335,  0.0426], grad_fn=<SelectBackward0>)

In [46]:
sum([-0.0323, -0.1437,  0.1171,  0.1633,  0.0516, -0.0692, -0.1154, -0.0357,
    0.0878,  0.0021, -0.0021,  0.2095, -0.0472, -0.1498, -0.0840,  0.0021,
    -0.0688,  0.0677, -0.0335,  0.0426])

-0.03790000000000001

Does this make sense to you? If you are unclear, go back to the classification problem with the MNIST, where we had 10 classes.
- No, because they don't sum to 1. It has to be 1 of the 20 classes so together the predicted values should be 1.

We have a classification problem, so we need Cross Entropy Loss.
Remember, [this has a softmax built in](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) 

In [47]:
loss_fn = torch.nn.CrossEntropyLoss()
loss = loss_fn(yhat, y)
loss

tensor(3.0137, grad_fn=<NllLossBackward0>)

In [48]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

# on my mac, at least for the BaseRNN model, mps does not speed up training
# probably because the overhead of copying the data to the GPU is too high
# so i override the device to cpu
device = "cpu"
# however, it might speed up training for larger models, with more parameters

using cpu


Set up the settings for the trainer and the different types of logging you want

In [57]:
settings = TrainerSettings(
    epochs=20, # increase this to about 100 for training
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": False, # save every best model, and restore the best one
        "verbose": True,
        "patience": 5, # number of epochs with no improvement after which training will be stopped
        "delta": 0.0, # minimum change to be considered an improvement
    }
)
settings

epochs: 20
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 5, 'delta': 0.0}

In [14]:
import torch.nn as nn
import torch
from torch import Tensor
from dataclasses import dataclass

@dataclass
class ModelConfig:
    input_size: int
    hidden_size: int
    num_layers: int
    output_size: int
    dropout: float = 0.0

class GRUmodel(nn.Module):
    def __init__(
        self,
        config,
    ) -> None:
        super().__init__()
        self.config = config
        self.rnn = nn.GRU(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.rnn(x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

In [58]:
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)

with mlflow.start_run():
    mlflow.set_tag("model", "gru-gestures_experiment_layers4_units512_epochs50")
    mlflow.set_tag("dev", "tyco")
    config = ModelConfig(
        input_size=3,
        hidden_size=512,
        num_layers=4,
        output_size=20,
        dropout=0.1,
    )

    model = GRUmodel(
        config=config,
    )

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
        device=device,
    )
    trainer.loop()

    if not settings.earlystop_kwargs["save"]:
        tag = datetime.now().strftime("%Y%m%d-%H%M-")
        modelpath = modeldir / (tag + "model.pt")
        torch.save(model, modelpath)

2025-10-14 16:21:02.608 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures\20251014-162102
2025-10-14 16:21:02.608 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [01:05<00:00,  1.23it/s]
2025-10-14 16:22:15.164 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.1949 test 1.5590 metric ['0.4437']
100%|██████████| 81/81 [01:28<00:00,  1.09s/it]
2025-10-14 16:23:50.055 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.0355 test 0.6412 metric ['0.8141']
100%|██████████| 81/81 [00:59<00:00,  1.36it/s]
2025-10-14 16:24:53.536 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.3547 test 0.2684 metric ['0.9250']
100%|██████████| 81/81 [00:56<00:00,  1.44it/s]
2025-10-14 16:25:53.203 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.1184 test 0.1734 metric ['0.9516']
100%|██████████| 81/81 [00:56<00:00,  1.44it/s]
2025-10-14 16:26:53.30

Try to update the code above by changing the hyperparameters.
- after testing with 3 epochs it looks like the best results happen with a not to deep (4 layers) and wider (512) network. After seeing this - I decided to crank up to epochs to 50 to see if further improvement of difficult patterns helps.
    
To discern between the changes, also modify the tag mlflow.set_tag("model", "new-tag-here") where you add
a new tag of your choice. This way you can keep the models apart.

In [17]:
trainer.loop() # if you want to pick up training, loop will continue from the last epoch

100%|██████████| 81/81 [00:02<00:00, 35.44it/s]
2025-10-13 21:24:14.932 | INFO     | mltrainer.trainer:report:175 - Resuming epochs from previous training at 3
2025-10-13 21:24:15.071 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 1.9470 test 1.8368 metric ['0.3359']
100%|██████████| 81/81 [00:02<00:00, 33.93it/s]
2025-10-13 21:24:17.673 | INFO     | mltrainer.trainer:report:209 - Epoch 4 train 1.7459 test 1.6587 metric ['0.4672']
100%|██████████| 81/81 [00:01<00:00, 41.21it/s]
2025-10-13 21:24:19.867 | INFO     | mltrainer.trainer:report:209 - Epoch 5 train 1.5600 test 1.4342 metric ['0.5203']
100%|██████████| 3/3 [00:07<00:00,  2.46s/it]


In [18]:
mlflow.end_run()

In [59]:
import torch.nn as nn
import torch
from torch import Tensor
from dataclasses import dataclass

@dataclass
class ModelConfig:
    input_size: int
    hidden_size: int
    num_layers: int
    output_size: int
    dropout: float = 0.0

class LSTMmodel(nn.Module):
    def __init__(
        self,
        config,
    ) -> None:
        super().__init__()
        self.config = config
        self.rnn = nn.LSTM(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.rnn(x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

In [60]:
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)

with mlflow.start_run():
    mlflow.set_tag("model", "lstm-gestures_experiment_layers4_units512_epochs20")
    mlflow.set_tag("dev", "tyco")
    config = ModelConfig(
        input_size=3,
        hidden_size=512,
        num_layers=4,
        output_size=20,
        dropout=0.1,
    )

    model = LSTMmodel(
        config=config,
    )

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
        device=device,
    )
    trainer.loop()

    if not settings.earlystop_kwargs["save"]:
        tag = datetime.now().strftime("%Y%m%d-%H%M-")
        modelpath = modeldir / (tag + "model.pt")
        torch.save(model, modelpath)

2025-10-15 17:36:45.994 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures\20251015-173645
2025-10-15 17:36:45.996 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:47<00:00,  1.69it/s]
2025-10-15 17:37:45.061 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.5746 test 2.4262 metric ['0.1313']
100%|██████████| 81/81 [01:31<00:00,  1.13s/it]
2025-10-15 17:39:19.242 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.2409 test 2.1812 metric ['0.2687']
100%|██████████| 81/81 [00:39<00:00,  2.06it/s]
2025-10-15 17:40:00.441 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 2.0486 test 2.7735 metric ['0.2437']
2025-10-15 17:40:00.441 | INFO     | mltrainer.trainer:__call__:252 - best loss: 2.1812, current loss 2.7735.Counter 1/5.
100%|██████████| 81/81 [00:34<00:00,  2.38it/s]
2025-10-15 17:40:36.289 | INFO     | mltrainer.trainer:report:209 -

Try another Rnn model

In [61]:
import mlflow
from datetime import datetime
from mltrainer.rnn_models import AttentionGRU

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)

with mlflow.start_run():
    mlflow.set_tag("model", "gruAttention-gestures_experiment_layers4_units512_epochs20")
    mlflow.set_tag("dev", "tyco")
    config = ModelConfig(
        input_size=3,
        hidden_size=512,
        num_layers=4,
        output_size=20,
        dropout=0.1,
    )

    model = AttentionGRU(
        config=config,
    )

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
        device=device,
    )
    trainer.loop()

    if not settings.earlystop_kwargs["save"]:
        tag = datetime.now().strftime("%Y%m%d-%H%M-")
        modelpath = modeldir / (tag + "model.pt")
        torch.save(model, modelpath)

2025-10-15 20:22:40.329 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures\20251015-202240
2025-10-15 20:22:40.331 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:59<00:00,  1.36it/s]
2025-10-15 20:23:44.199 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.1427 test 1.3564 metric ['0.5422']
100%|██████████| 81/81 [01:00<00:00,  1.34it/s]
2025-10-15 20:24:49.321 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.8122 test 0.5733 metric ['0.8547']
100%|██████████| 81/81 [01:00<00:00,  1.33it/s]
2025-10-15 20:25:58.677 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.5179 test 0.3336 metric ['0.9125']
100%|██████████| 81/81 [01:46<00:00,  1.32s/it]
2025-10-15 20:27:49.687 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.1866 test 0.1380 metric ['0.9688']
100%|██████████| 81/81 [01:07<00:00,  1.21it/s]
2025-10-15 20:29:01.78

Experiment with conv1d layer